In [ ]:
!pip install PyMuPDF pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.3 MB/s eta 0:00:00


In [ ]:
import pdfplumber

# Load the PDF
pdf_path = '/content/Your Guide to A Healthy Heart Autor Marian Sandmaier.pdf'  # Path to your PDF file
with pdfplumber.open(pdf_path) as pdf:
    pdf_text = []
    for page in pdf.pages:
        text = page.extract_text()
        pdf_text.append(text)

# pdf_text will now hold a list of texts per page


In [ ]:
pdf_text[1:5]

['Y O U R G U I D E T O\nA Healthy Heart\nU.S. DEPARTMENTOFHEALTHANDHUMANSERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute\nNIH Publication No. 06-5269\nDecember 2005',
 'Written by: Marian Sandmaier\nU.S. DEPARTMENT OF HEALTH AND HUMAN SERVICES\nNational Institutes of Health\nNational Heart, Lung, and Blood Institute',
 'Heart Disease: Why Should You Care?. . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nWhat You Need To Know About Heart Disease. . . . . . . . . . . . . . . . . . . . . . 3\nWhat Is Heart Disease?. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3\nWho Is at Risk?. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\nHow Risk Works . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\nWhat’s Your Risk? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nHow To Talk With Your Docto

In [ ]:
def chunk_text(text, max_length=500):
    words = text.split()
    chunks = [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]
    return chunks

# Chunk the entire PDF text
knowledge_chunks = []
for page_text in pdf_text:
    knowledge_chunks.extend(chunk_text(page_text))


In [ ]:
!pip install torch faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load a sentence embedding model (e.g., MiniLM)
encoder = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
encoder_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Encode the text chunks
def encode_chunks(chunks):
    inputs = encoder_tokenizer(chunks, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = encoder(**inputs).last_hidden_state.mean(dim=1)  # Get the average of token embeddings
    return embeddings

chunk_embeddings = encode_chunks(knowledge_chunks)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import faiss

# Initialize a FAISS index
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])  # L2 distance for similarity search
index.add(chunk_embeddings.cpu().numpy())  # Add the chunk embeddings to the index


In [ ]:
def retrieve_pdf_passages(query, top_k=3):
    # Encode the query
    query_embedding = encode_chunks([query]).cpu().numpy()

    # Search in FAISS index
    _, indices = index.search(query_embedding, top_k)
    return [knowledge_chunks[i] for i in indices[0]]  # Return the top-k retrieved passages


In [ ]:
query = "What are the causes of cardiovascular disease?"
retrieved_chunks = retrieve_pdf_passages(query)
print("Retrieved Chunks:", retrieved_chunks)

Retrieved Chunks: ['4 Who Is at Risk? Risk factors are conditions or habits that make a person more likely to develop a disease. They can also increase the chances that an existing disease will get worse. Important risk factors for heart dis- ease that you can do something about are cigarette smoking, high blood pressure, high blood cholesterol, overweight, physical inactivity, and diabetes. Recent research shows that more than 95 percent of those who die from heart disease have at least one of these major risk factors. Certain risk factors, such as getting older, can’t be changed. After menopause, women are more likely to develop heart disease. For both women and men, middle age is a time of increasing risk because people are more likely to develop heart disease risk factors during this stage of life. Family history of early heart disease is another risk factor that can’t be changed. If your father or brother had a heart attack before age 55, or if your mother or sister had one before

In [ ]:
from google.colab import output
output.set_max_output_height(0)  # Setting it to 0 removes the limit

AttributeError: module 'google.colab.output' has no attribute 'set_max_output_height'

In [ ]:
from IPython.display import display, HTML

display(HTML('''
    <style>
    .output_scroll {
        height: auto !important;
        max-height: none !important;
    }
    </style>
'''))


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
def generate_answer_from_pdf(query, retrieved_chunks):
    # Combine the query with the retrieved passages
    input_text = query + " " + " ".join(retrieved_chunks)
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)

    # Generate the answer
    output = model.generate(**inputs, max_length=150, num_beams=3, early_stopping=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
answer = generate_answer_from_pdf(query, retrieved_chunks)
print("Generated Answer:", answer)


Generated Answer: What are the causes of cardiovascular disease? 4 Who Is at Risk? Risk factors are conditions or habits that make a person more likely to develop a disease. They can also increase the chances that an existing disease will get worse. For example, if you have high blood cholesterol and you smoke, your heart disease risk increases enormously. While each risk factor increases your risk of heart disease, having more than one risk factor is especially serious. That’s because risk factors tend to “gang up” and worsen each other’m effects. To protect your heart, it is vital to make changes that address each and every risk factor you have. How Risk Works It may be tempting to believe that doing just one healthy thing will


## Use standard textbooks and optimized model